In [106]:
import re, json, numpy as np, pandas as pd
from pathlib import Path
from typing import Dict, List
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score, confusion_matrix
from scipy.sparse import hstack
import joblib

# Конфиг пути и тд

In [107]:
DATA_CSV = "/content/L_corrected.csv"      # колонки: text,label
TEXT_COL = "Desc"
LABEL_COL = "Group"
MAJOR_CLASS = "Управление домом"              # мажорный класс (переименуй под свои данные)
OUT_DIR = Path("artifacts"); OUT_DIR.mkdir(exist_ok=True)

# Лексиконы
- Лексиконы - ключевые слова (правила) по которым определяем какие классы какой службе соответствуют.
- Это не жесткие правила а лишь помощь классификатору

In [108]:
LEX: Dict[str, List[str]] = {
    # Канализация/стояки/запахи/засоры
    "Канализация": [
        r"\bканализац", r"\bзасор", r"\bпрочистк", r"\bколодец", r"\bстояк", r"\bсифон",
        r"\bунитаз", r"\bраковин", r"\bслив", r"\bфанова", r"\bзапах(?!.*газа)", r"\bжироулав", r"\bкрышка колодца"
    ],

    # Отвод воды/КНС/ливнёвка/затопления подвала
    "Водоотведение": [
        r"\bводоотвед", r"\bкнс\b", r"\bливнев", r"\bдождепри", r"\bстоки", r"\bфекаль",
        r"\bзатоп(ил[аи]|ило)\b", r"\bподвале\b.*\bвода\b", r"\bперелив"
    ],

    # Электрика: отсутствие света, пробки/автоматы, щиток, КЗ, искрит, освещение
    "Электроэнергия": [
        r"\bнет\s+света", r"\bвыбил[аи].*\bпробк", r"\bпробк[аи]\b", r"\bавтомат(ы)?\b", r"\bщиток",
        r"\bзамыкан", r"\bкоротит", r"\bискрит", r"\bпровод", r"\bрозетк", r"\bосвещен", r"\bламп(а|ы|очек|очку)"
    ],

    # ГВС: горячая вода и счётчики/опломбировка/циркуляция
    "ГВС": [
        r"\bгвс\b", r"\bгоряч(ая|ей)\s+вод", r"\bнет\s+горяч", r"\bтемператур", r"\bсчетчик", r"\bопломб",
        r"\bпломб", r"\bподмес", r"\bциркуляц"
    ],

    # Лифты: лифт, застрял, не работает, двери, кнопка вызова, кабина, шумы
    "Лифты": [
        r"\bлифт", r"\bзастрял", r"\bзастряли", r"\bне\s+работает\s+лифт", r"\bдвер(ь|и)\s+(не\s+)?(закрыва|открыва)",
        r"\bкнопк[аи]\s+вызова", r"\bкабин[ае]", r"\bскрежет", r"\bреверс"
    ],

    # Домофоны: домофон, трубка, панель, ключ/магнит, не открывает дверь
    "Домофоны": [
        r"\bдомофон", r"\bтрубк", r"\bпанел", r"\bвызов", r"\bне\s+работает\s+домофон", r"\bключ(и|ей)\b",
        r"\bмагнит", r"\bдвер(ь|и)\b.*\bне\s+открыва", r"\bкод(ы)?", r"\bзвонок"
    ],

    # Управление домом / ОДУ: подъезд, двор, уборка, кровля/крыша и пр. общее
    "Управление домом": [
        r"\bподъезд", r"\bдвор", r"\bуборк[аи]", r"\bмусор", r"\bснег", r"\bналедь|\bгололед", r"\bкровл|крыша",
        r"\bдвер(ь|и)\s+подъезда", r"\bперила", r"\bпочистить"
    ],

    # Редкие, но полезные — можно оставить, если есть такие лейблы
    "Отопление": [
        r"\bнет\s+отоплен", r"\bбатаре(я|и)\s+(холод|лед)", r"\bрадиатор", r"\bстояк\s+отоплен", r"\bкотел"
    ],
    "Газоснабжение": [
        r"\bгаз\b", r"\bзапах\s+газа", r"\bутечк[аи]\s+газа", r"\bгазов", r"\bпломб.*газ"
    ],
}


# МАШИН ЛЕРНИНГ!!!

In [109]:
def normalize_text(s: str) -> str:
    s = s.lower()
    s = re.sub(r"\s+", " ", s).strip()
    return s

## `build_rule_feats(texts: List[str], classes: List[str]) -> np.ndarray`

**Назначение:**  
Генерирует дополнительные "правильные" (rule-based) признаки для каждого текста на основе заранее заданных словарей/регулярных выражений (`LEX`).  
Каждый признак — это количество совпадений текста с шаблонами, характерными для определённого класса.

**Параметры:**
- `texts` (`List[str]`): Список текстовых обращений пользователей.
- `classes` (`List[str]`): Список всех классов (меток), для которых будут считаться совпадения.

**Возвращает:**
- `np.ndarray` формы `(N, C)`:
  - `N` — количество текстов,
  - `C` — количество классов.  
  Каждое значение `M[i, j]` — количество паттернов из словаря класса `classes[j]`, найденных в тексте `texts[i]`.

**Алгоритм работы:**
1. Для каждого класса из `classes` берутся регулярные выражения из глобального словаря `LEX` и компилируются для ускорения поиска.
2. Создаётся матрица признаков `M` из нулей (`float32`).
3. Для каждого текста и каждого класса считается, сколько паттернов найдено в тексте.
4. Результирующая матрица возвращается в виде `numpy.ndarray`.

**Пример:**
```python
texts = ["Прорвало трубу в подвале", "Выбило пробки"]
classes = ["Сантехника", "Электрика"]

M = build_rule_feats(texts, classes)
# M[0,0] может быть >0 (нашли слова про трубы)
# M[0,1] = 0 (нет электрических терминов)
```

In [110]:
def build_rule_feats(texts: List[str], classes: List[str]):
    # матрица N x C: сколько паттернов класса сработало в тексте
    comp = {c: [re.compile(p, re.I) for p in LEX.get(c, [])] for c in classes}
    M = np.zeros((len(texts), len(classes)), dtype=np.float32)
    for i, t in enumerate(texts):
        for j, c in enumerate(classes):
            if comp[c]:
                M[i, j] = sum(1 for pat in comp[c] if pat.search(t))
    return M

In [111]:
df = pd.read_csv(DATA_CSV)
df = df[[TEXT_COL, LABEL_COL]].dropna()
df[TEXT_COL] = df[TEXT_COL].astype(str).map(normalize_text)

In [112]:
df['Group'].value_counts()

,count
Group,
Управление домом,24081
Канализация,12487
Водоотведение,5428
Электроэнергия,5027
ГВС,1729
Лифты,1269
Домофоны,859
Благоустройство,58
Общие вопросы,50


In [113]:
X_train, X_val, y_train, y_val = train_test_split(
    df[TEXT_COL].values, df[LABEL_COL].values, test_size=0.2, stratify=df[LABEL_COL], random_state=42
)

In [114]:
# === 1) TF-IDF по словам (1-2 граммы) ===
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=3, max_df=0.9)
Xtr_tfidf = tfidf.fit_transform(X_train)
Xva_tfidf = tfidf.transform(X_val)

In [115]:
# === 2) Rule-feats ===
classes_sorted = sorted(pd.unique(df[LABEL_COL]))
Xtr_rule = build_rule_feats(list(X_train), classes_sorted)
Xva_rule = build_rule_feats(list(X_val), classes_sorted)
# совместим sparse + dense
Xtr = hstack([Xtr_tfidf, Xtr_rule])
Xva = hstack([Xva_tfidf, Xva_rule])

In [126]:
# === 3) Веса классов и/или sample_weight ===
sample_weight = np.ones(len(y_train), dtype=np.float32)
sample_weight[np.array(y_train) == MAJOR_CLASS] = 0.6

In [127]:
clf = LogisticRegression(
        max_iter=2000, class_weight="balanced", n_jobs=-1, solver="lbfgs", multi_class="auto"
    )
clf.fit(Xtr, y_train, sample_weight=sample_weight)


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(class_weight='balanced', max_iter=2000, multi_class='auto',
                   n_jobs=-1)

In [128]:
proba = clf.predict_proba(Xva)
y_pred_plain = clf.classes_[proba.argmax(1)]

In [129]:
print("\n=== Без порога ===")
print(classification_report(y_val, y_pred_plain, digits=3))
print("Confusion matrix:\n", confusion_matrix(y_val, y_pred_plain, labels=clf.classes_))


=== Без порога ===
                  precision    recall  f1-score   support

 Благоустройство      0.039     0.167     0.063        12
   Водоотведение      0.931     0.975     0.953      1086
             ГВС      0.884     0.991     0.935       346
        Домофоны      0.746     0.971     0.843       172
     Канализация      0.987     0.924     0.955      2497
           Лифты      0.922     0.972     0.946       254
   Общие вопросы      0.154     0.400     0.222        10
Управление домом      0.964     0.937     0.951      4816
  Электроэнергия      0.910     0.980     0.943      1005

        accuracy                          0.944     10198
       macro avg      0.726     0.813     0.757     10198
    weighted avg      0.951     0.944     0.947     10198

Confusion matrix:
 [[   2    3    1    0    1    1    0    3    1]
 [   2 1059    0    0    6    0    1   18    0]
 [   0    0  343    0    2    0    1    0    0]
 [   0    0    0  167    0    0    0    3    2]
 [   7    8 

# Экспорт модели

In [121]:
joblib.dump({"tfidf": tfidf, "clf": clf, "classes": clf.classes_,
             "major_class": MAJOR_CLASS, "lex": LEX},
            OUT_DIR / "service_clf.joblib")
print("\nSaved:", OUT_DIR / "service_clf.joblib")


Saved: artifacts/service_clf.joblib
